In [1]:
from transformers import * 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import random 
import re 
import math 
from tqdm import tqdm 
import sklearn 
import torch 
import torch.nn as nn 
import torch.functional as f
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from sklearn.model_selection import train_test_split 
import time 
import datetime 

In [2]:
!nvidia-smi

Sun Jul 18 14:18:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   33C    P0    44W / 300W |      0MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv') 

train.shape, test.shape, train['label'].nunique() 

((174304, 13), (43576, 12), 46)

In [4]:
num_classes = train['label'].nunique() # 46

In [5]:
submission = pd.read_csv("sample_submission.csv") 

In [6]:
def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

In [7]:
def split_text(s, overlap = 20, chunk_size = 50): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0:  
        n = len(s.split()) // (chunk_size - overlap) 
    else: 
        n = 1 
    for w in range(n): 
        if w == 0: 
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else:  
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size]
            total.append(" ".join(partial)) 
    return total


In [8]:
train['요약문_내용'] = train['요약문_연구목표'] + train['요약문_연구내용'] + train['요약문_기대효과'] 
test['요약문_내용'] = test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과']

In [9]:
train['요약문_내용'].fillna('NAN',inplace=True) 
test['요약문_내용'].fillna('NAN',inplace=True)

In [10]:
train['사업명'].fillna('NAN',inplace=True) 
train['사업_부처명'].fillna('NAN',inplace=True) 
train['내역사업명'].fillna('NAN',inplace=True) 
train['과제명'].fillna('NAN',inplace=True) 
train['요약문_한글키워드'].fillna('NAN',inplace=True)

In [11]:
contents = train['요약문_내용'].values 
feature1 = train['사업명'].values 
feature2 = train['사업_부처명'].values 
feature3 = train['내역사업명'].values 
feature4 = train['과제명'].values 
feature5 = train['요약문_한글키워드'].values 
feature6 = train['label'].values 

train_data = {'사업명':[],'사업_부처명':[],'내역사업명':[],'과제명':[],'한글키워드':[],'요약문':[],'label':[]} 

for i in tqdm(range(contents.shape[0]), position = 0, leave = True): 
    sample = str(contents[i]) 
    splitted_text = split_text(clean_text(sample)) 
    for t in splitted_text: 
        train_data['요약문'].append(t) 
        train_data['사업명'].append(clean_text(str(feature1[i])))
        train_data['사업_부처명'].append(clean_text(str(feature2[i]))) 
        train_data['내역사업명'].append(clean_text(str(feature3[i]))) 
        train_data['과제명'].append(clean_text(str(feature4[i])))  
        train_data['한글키워드'].append(feature5[i]) # no cleaning for this one
        train_data['label'].append(feature6[i])  

100%|██████████| 174304/174304 [01:29<00:00, 1947.81it/s]


In [12]:
train_data = pd.DataFrame(train_data)

In [13]:
train_data.head(5)

,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24
2,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",최종목표 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구 발굴된 유전자를 통한 ...,0
3,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",저항성 극복 기전을 규명 환자조직 동물실험 세포실험을 통해 대장암에 특이적으로 조절...,0
4,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",내성 유발 유전자를 발굴하고 저항성 극복 기전을 규명 추후 기반 항암화학요법 치료효...,0


In [14]:
## Now we tokenize each data and make sure they all lie within the 512 tokenization range 
## if not check how many have token length greater than 512 

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                     bos_token='</s>',
                                                     eos_token='</s>',
                                                     unk_token='<unk>',
                                                     pad_token='<pad>',
                                                     mask_token='<mask>') 

 

def gpt2_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences, not "really" necessary for now    
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        token_type_id = token_type_id[:129] + token_type_id[-383:]    
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return input_id, attention_mask, token_type_id


In [15]:
tokenizer.padding_side = 'left'

In [16]:
train_data['data'] = train_data['사업명'] + " " + train_data['사업_부처명'] + " " + train_data['내역사업명'] + " " + train_data['과제명'] + " " + train_data['한글키워드'] + " " + train_data['요약문'] 

train_data.head(2)


,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label,data
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...


In [17]:
train_text = train_data['data'].values 
train_labels = train_data['label'].values

In [19]:
BATCH_SIZE = 8
NUM_EPOCHS = 20
VALID_SPLIT = 0.1 
MAX_LEN = 512 # max token size for BERT, ELECTRA, GPT2

In [21]:
N = train_data.shape[0] 

input_ids = np.zeros((N, MAX_LEN),dtype=int)
attention_masks = np.zeros((N, MAX_LEN),dtype=int)
labels = np.zeros((N),dtype=int)

for i in tqdm(range(N), position=0, leave=True): 
    try:
        cur_str = train_text[i]
        cur_label = train_labels[i]
        input_id, attention_mask = gpt2_tokenizer(cur_str, MAX_LEN=MAX_LEN) 
        input_ids[i,] = input_id 
        attention_masks[i,] = attention_mask 
        labels[i] = cur_label 
    except Exception as e: 
        print(e)
        print(cur_str)
        pass


 11%|█         | 173353/1638867 [01:27<12:25, 1966.78it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 14%|█▍        | 234077/1638867 [01:59<12:13, 1914.28it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 18%|█▊        | 292414/1638867 [02:29<11:50, 1896.35it/s]

...

 77%|███████▋  | 1258048/1638867 [11:07<03:39, 1733.44it/s]

Long Text!! Using Head+Tail Truncation


 83%|████████▎ | 1353266/1638867 [12:00<02:47, 1709.28it/s]

Long Text!! Using Head+Tail Truncation


100%|██████████| 1638867/1638867 [14:42<00:00, 1857.23it/s]


In [23]:
input_ids = torch.tensor(input_ids, dtype=int)

In [24]:
attention_masks = torch.tensor(attention_masks, dtype=int)

In [25]:
labels = torch.tensor(labels, dtype=int)

In [26]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, random_state = 42, test_size = VALID_SPLIT) 

train_attention_mask, val_attention_mask, _, _ = train_test_split(attention_masks, labels, random_state = 42, test_size = VALID_SPLIT) 


train_inputs.shape, train_attention_mask.shape, train_labels.shape


(torch.Size([1474980, 512]), torch.Size([1474980, 512]), torch.Size([1474980]))

In [27]:
val_inputs.shape, val_attention_mask.shape, val_labels.shape

(torch.Size([163887, 512]), torch.Size([163887, 512]), torch.Size([163887]))

In [28]:
train_data = TensorDataset(train_inputs, train_attention_mask, train_labels) 
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE) 

validation_data = TensorDataset(val_inputs, val_attention_mask, val_labels) 
validation_sampler = SequentialSampler(validation_data) 
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)


# Define GPT2 model

In [29]:
model = GPT2ForSequenceClassification.from_pretrained("skt/kogpt2-base-v2", num_labels=46) 
model.cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=Fal

In [34]:
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id 


In [35]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8) 

epochs = 20

total_steps = len(train_dataloader) * epochs 

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [36]:
def flat_accuracy(preds, labels): 
    pred_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten() 
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [37]:
device = torch.device('cuda')

In [38]:
# 재현을 위해 랜덤시드 고정
device = torch.device("cuda")
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  current average loss = {}'.format(total_loss / step))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels = b_labels)
    
        
        loss = outputs[0] 
        logits = outputs[1] 
        
        # 로스 구함 
        eval_loss += loss.item() 
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    avg_val_loss = eval_loss / len(validation_dataloader)            
    print("  Average validation loss: {0:.2f}".format(avg_val_loss))
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    torch.save(model.state_dict(), "KoGPT2_voting_" + str(epoch_i + 1)) 
print("")
print("Training complete!")



======== Epoch 1 / 20 ========
Training...
  Batch   100  of  184,373.    Elapsed: 0:00:31.
  current average loss = 1.0278879571706057
  Batch   200  of  184,373.    Elapsed: 0:01:01.
  current average loss = 0.9854144696099684
  Batch   300  of  184,373.    Elapsed: 0:01:31.
  current average loss = 0.9561178203276358
  Batch   400  of  184,373.    Elapsed: 0:02:01.
  current average loss = 0.9179535548150306
  Batch   500  of  184,373.    Elapsed: 0:02:30.
  current average loss = 0.8904824114185758
  Batch   600  of  184,373.    Elapsed: 0:03:00.
  current average loss = 0.8478580296097789
  Batch   700  of  184,373.    Elapsed: 0:03:30.
  current average loss = 0.8206086225509976
  Batch   800  of  184,373.    Elapsed: 0:03:59.
  current average loss = 0.8031867989593593
  Batch   900  of  184,373.    Elapsed: 0:04:29.
  current average loss = 0.7777667351479693
  Batch 1,000  of  184,373.    Elapsed: 0:04:59.
  current average loss = 0.7670043660182273
  Batch 1,100  of  184,373

  Batch 8,900  of  184,373.    Elapsed: 0:44:10.
  current average loss = 0.5107704440222569
  Batch 9,000  of  184,373.    Elapsed: 0:44:39.
  current average loss = 0.5099476752288117
  Batch 9,100  of  184,373.    Elapsed: 0:45:09.
  current average loss = 0.5090986489556331
  Batch 9,200  of  184,373.    Elapsed: 0:45:38.
  current average loss = 0.5081369727112913
  Batch 9,300  of  184,373.    Elapsed: 0:46:08.
  current average loss = 0.5073150701638506
  Batch 9,400  of  184,373.    Elapsed: 0:46:38.
  current average loss = 0.5067433027913609
  Batch 9,500  of  184,373.    Elapsed: 0:47:07.
  current average loss = 0.5056487130538452
  Batch 9,600  of  184,373.    Elapsed: 0:47:37.
  current average loss = 0.5048083631652276
  Batch 9,700  of  184,373.    Elapsed: 0:48:06.
  current average loss = 0.5048421665370927
  Batch 9,800  of  184,373.    Elapsed: 0:48:36.
  current average loss = 0.5037834942780148
  Batch 9,900  of  184,373.    Elapsed: 0:49:05.
  current average los

  Batch 17,900  of  184,373.    Elapsed: 1:28:32.
  current average loss = 0.4510460505111576
  Batch 18,000  of  184,373.    Elapsed: 1:29:02.
  current average loss = 0.451139321460161
  Batch 18,100  of  184,373.    Elapsed: 1:29:32.
  current average loss = 0.4506535992414282
  Batch 18,200  of  184,373.    Elapsed: 1:30:01.
  current average loss = 0.44984808049833663
  Batch 18,300  of  184,373.    Elapsed: 1:30:31.
  current average loss = 0.44915936079156044
  Batch 18,400  of  184,373.    Elapsed: 1:31:00.
  current average loss = 0.44843268635956285
  Batch 18,500  of  184,373.    Elapsed: 1:31:30.
  current average loss = 0.44813980836569955
  Batch 18,600  of  184,373.    Elapsed: 1:32:00.
  current average loss = 0.44749000834433844
  Batch 18,700  of  184,373.    Elapsed: 1:32:29.
  current average loss = 0.446644121217092
  Batch 18,800  of  184,373.    Elapsed: 1:32:59.
  current average loss = 0.44645051487886256
  Batch 18,900  of  184,373.    Elapsed: 1:33:28.
  curr

  Batch 26,600  of  184,373.    Elapsed: 2:11:26.
  current average loss = 0.4193879890069989
  Batch 26,700  of  184,373.    Elapsed: 2:11:56.
  current average loss = 0.41918526154163765
  Batch 26,800  of  184,373.    Elapsed: 2:12:26.
  current average loss = 0.41900698755910243
  Batch 26,900  of  184,373.    Elapsed: 2:12:55.
  current average loss = 0.4187643445321875
  Batch 27,000  of  184,373.    Elapsed: 2:13:25.
  current average loss = 0.41857714296441584
  Batch 27,100  of  184,373.    Elapsed: 2:13:54.
  current average loss = 0.4182789005330307
  Batch 27,200  of  184,373.    Elapsed: 2:14:24.
  current average loss = 0.41795751169164386
  Batch 27,300  of  184,373.    Elapsed: 2:14:54.
  current average loss = 0.41743558840077344
  Batch 27,400  of  184,373.    Elapsed: 2:15:23.
  current average loss = 0.4172091726217305
  Batch 27,500  of  184,373.    Elapsed: 2:15:53.
  current average loss = 0.4168832330445796
  Batch 27,600  of  184,373.    Elapsed: 2:16:22.
  cur

  Batch 35,300  of  184,373.    Elapsed: 2:54:21.
  current average loss = 0.395961116617556
  Batch 35,400  of  184,373.    Elapsed: 2:54:51.
  current average loss = 0.39549256098109387
  Batch 35,500  of  184,373.    Elapsed: 2:55:20.
  current average loss = 0.39527466963600627
  Batch 35,600  of  184,373.    Elapsed: 2:55:49.
  current average loss = 0.39498489009513005
  Batch 35,700  of  184,373.    Elapsed: 2:56:19.
  current average loss = 0.3947270017973149
  Batch 35,800  of  184,373.    Elapsed: 2:56:48.
  current average loss = 0.3946761438648555
  Batch 35,900  of  184,373.    Elapsed: 2:57:18.
  current average loss = 0.3943168188103294
  Batch 36,000  of  184,373.    Elapsed: 2:57:47.
  current average loss = 0.39388814160801505
  Batch 36,100  of  184,373.    Elapsed: 2:58:17.
  current average loss = 0.3938466501672316
  Batch 36,200  of  184,373.    Elapsed: 2:58:46.
  current average loss = 0.39361928068273033
  Batch 36,300  of  184,373.    Elapsed: 2:59:15.
  curr

...

  Batch 170,200  of  184,373.    Elapsed: 14:00:17.
  current average loss = 0.023318166552564804
  Batch 170,300  of  184,373.    Elapsed: 14:00:47.
  current average loss = 0.023307853139955252
  Batch 170,400  of  184,373.    Elapsed: 14:01:16.
  current average loss = 0.023321527156055167
  Batch 170,500  of  184,373.    Elapsed: 14:01:46.
  current average loss = 0.02331418719660808
  Batch 170,600  of  184,373.    Elapsed: 14:02:15.
  current average loss = 0.023323230031424835
  Batch 170,700  of  184,373.    Elapsed: 14:02:45.
  current average loss = 0.023332269633686415
  Batch 170,800  of  184,373.    Elapsed: 14:03:14.
  current average loss = 0.02333795490217562
  Batch 170,900  of  184,373.    Elapsed: 14:03:44.
  current average loss = 0.023331663937005746
  Batch 171,000  of  184,373.    Elapsed: 14:04:14.
  current average loss = 0.02332842489754927
  Batch 171,100  of  184,373.    Elapsed: 14:04:44.
  current average loss = 0.023347006112512825
  Batch 171,200  of  18

  Batch 178,600  of  184,373.    Elapsed: 14:41:49.
  current average loss = 0.023262623183755247
  Batch 178,700  of  184,373.    Elapsed: 14:42:19.
  current average loss = 0.023262066868567393
  Batch 178,800  of  184,373.    Elapsed: 14:42:49.
  current average loss = 0.02325765251664246
  Batch 178,900  of  184,373.    Elapsed: 14:43:18.
  current average loss = 0.02325765416917891
  Batch 179,000  of  184,373.    Elapsed: 14:43:48.
  current average loss = 0.023258173512407047
  Batch 179,100  of  184,373.    Elapsed: 14:44:18.
  current average loss = 0.02325749862366325
  Batch 179,200  of  184,373.    Elapsed: 14:44:47.
  current average loss = 0.023251108431946065
  Batch 179,300  of  184,373.    Elapsed: 14:45:17.
  current average loss = 0.02324612981625103
  Batch 179,400  of  184,373.    Elapsed: 14:45:48.
  current average loss = 0.023253806916951645
  Batch 179,500  of  184,373.    Elapsed: 14:46:19.
  current average loss = 0.02324097534003719
  Batch 179,600  of  184,

  Batch 2,600  of  184,373.    Elapsed: 0:12:50.
  current average loss = 0.019691445573184488
  Batch 2,700  of  184,373.    Elapsed: 0:13:20.
  current average loss = 0.01919771304997136
  Batch 2,800  of  184,373.    Elapsed: 0:13:50.
  current average loss = 0.019216604438973453
  Batch 2,900  of  184,373.    Elapsed: 0:14:19.
  current average loss = 0.019421494515608515
  Batch 3,000  of  184,373.    Elapsed: 0:14:49.
  current average loss = 0.019438103767524004
  Batch 3,100  of  184,373.    Elapsed: 0:15:18.
  current average loss = 0.019714941995324108
  Batch 3,200  of  184,373.    Elapsed: 0:15:49.
  current average loss = 0.019811212613956006
  Batch 3,300  of  184,373.    Elapsed: 0:16:19.
  current average loss = 0.0197578901554692
  Batch 3,400  of  184,373.    Elapsed: 0:16:48.
  current average loss = 0.0194721571108469
  Batch 3,500  of  184,373.    Elapsed: 0:17:18.
  current average loss = 0.01947856110894709
  Batch 3,600  of  184,373.    Elapsed: 0:17:48.
  curre

  Batch 11,300  of  184,373.    Elapsed: 0:55:52.
  current average loss = 0.01966463046393715
  Batch 11,400  of  184,373.    Elapsed: 0:56:21.
  current average loss = 0.019663965943484314
  Batch 11,500  of  184,373.    Elapsed: 0:56:51.
  current average loss = 0.019714889561800127
  Batch 11,600  of  184,373.    Elapsed: 0:57:21.
  current average loss = 0.01969115811076561
  Batch 11,700  of  184,373.    Elapsed: 0:57:50.
  current average loss = 0.019529608151579784
  Batch 11,800  of  184,373.    Elapsed: 0:58:20.
  current average loss = 0.019379154991020407
  Batch 11,900  of  184,373.    Elapsed: 0:58:50.
  current average loss = 0.01936359162867996
  Batch 12,000  of  184,373.    Elapsed: 0:59:19.
  current average loss = 0.01935724086955545
  Batch 12,100  of  184,373.    Elapsed: 0:59:49.
  current average loss = 0.019282593238645847
  Batch 12,200  of  184,373.    Elapsed: 1:00:19.
  current average loss = 0.019165231949774996
  Batch 12,300  of  184,373.    Elapsed: 1:0

KeyboardInterrupt: 

# Preprocess Test Data and Predict

We proceed with voting 

This time around, we will do a large ensemble of KoELECTRA, GPT2 and XLM-RoBERTa 

In [39]:
## load gpt2 
gpt2_checkpoint = torch.load("KoGPT2_voting_7")
test_gpt2 = GPT2ForSequenceClassification.from_pretrained("skt/kogpt2-base-v2", num_labels=46) 
test_gpt2.load_state_dict(gpt2_checkpoint)
test_gpt2.cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=Fal

In [55]:
# need to configure the model for GPT2 because we use the very last token
## Now we tokenize each data and make sure they all lie within the 512 tokenization range 
## if not check how many have token length greater than 512 

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                     bos_token='</s>',
                                                     eos_token='</s>',
                                                     unk_token='<unk>',
                                                     pad_token='<pad>',
                                                     mask_token='<mask>') 

 

def gpt2_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences, not "really" necessary for now    
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        token_type_id = token_type_id[:129] + token_type_id[-383:]    
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return input_id, attention_mask


In [57]:

tokenizer.padding_side = 'left'
test_gpt2.resize_token_embeddings(len(tokenizer))
test_gpt2.config.pad_token_id = tokenizer.pad_token_id 



In [43]:
## load Electra model 
electra_checkpoint = torch.load("ELECTRA_MORE_FEATURES_6") 
test_electra = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels = 46)
test_electra.load_state_dict(electra_checkpoint) 
test_electra.cuda() 

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [44]:
## load XLM-Roberta 
roberta_checkpoint = torch.load("XLM_ROBERTA_BASE_voting_6")
test_roberta = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=46) 
test_roberta.load_state_dict(roberta_checkpoint) 
test_roberta.cuda() 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [46]:
# define electra and roberta tokenizers 
# define Ko-ELECTRA tokenizer 
tokenizer_electra = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator") 

def electra_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer_electra.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences, not "really" necessary for now    
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        token_type_id = token_type_id[:129] + token_type_id[-383:]    
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return input_id, attention_mask, token_type_id

In [47]:
# define XLM-RoBERTa tokenizer
## Now we tokenize each data and make sure they all lie within the 512 tokenization range 
## if not check how many have token length greater than 512 

tokenizer_roberta = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base") 

def roberta_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer_roberta.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        
    return input_id, attention_mask 


In [45]:
test['요약문_내용'] = test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과'] 
test['요약문_내용'].fillna('NAN',inplace=True)

In [48]:
test_contents = test['요약문_내용'].values 
test_feature1 = test['사업명'].values 
test_feature2 = test['사업_부처명'].values 
test_feature3 = test['내역사업명'].values 
test_feature4 = test['과제명'].values 
test_feature5 = test['요약문_한글키워드'].values 
test_feature6 = test['요약문_영문키워드'].values 

In [ ]:
import torch.nn.functional as nnf

predicted_classes = []

# change to eval mode 
test_electra.eval() 
test_roberta.eval() 
test_gpt2.eval() 


for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i])  
    f6 = str(test_feature6[i]) 
    splitted = split_text(clean_text(str(test_contents[i]))) 
    # make predictions for each splitted text 
    probabilities_electra = [] # store probabilities for electra 
    probabilities_roberta = [] # store probabilities for xlm-roberta  
    probabilities_gpt2 = [] # store probabilities for gpt2 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + f6 + " " + text 
        ''' XLM-RoBERTa predictions '''
        # tokenize test text 
        input_id, attention_mask = roberta_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1,MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1,MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        
        with torch.no_grad(): 
            outputs = test_roberta(input_id, 
                                   token_type_ids=None, 
                                   attention_mask=attention_mask) 
        
        logits = outputs[0]
        
        # obtain softmax probabilities 
        prob = nnf.softmax(logits, dim=1).flatten()
        probabilities_roberta.append(prob)

        ''' KoELECTRA predictions ''' 
        # tokenize test text 
        input_id, attention_mask, token_type_ids = electra_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        token_type_ids = torch.tensor(token_type_ids) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1, MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1, MAX_LEN)) 
        token_type_ids = torch.reshape(token_type_ids, (-1, MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        token_type_ids = token_type_ids.to(device) 
        with torch.no_grad(): 
            outputs = test_electra(input_id, 
                                   token_type_ids=token_type_ids,
                                   attention_mask = attention_mask)

        logits = outputs[0]
        
        # obtain softmax probabilities  
        prob = nnf.softmax(logits, dim=1).flatten() 
        probabilities_electra.append(prob)  
        
        ''' GPT2 predictions ''' 
        # tokenize test text 
        input_id, attention_mask = gpt2_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id) 
        attention_mask = torch.tensor(attention_mask) 
        # reshape into (batch, MAX_LEN) 
        input_id = torch.reshape(input_id, (-1, MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1,MAX_LEN))
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        with torch.no_grad(): 
            outputs = test_gpt2(input_id, 
                                token_type_ids=None,
                                attention_mask = attention_mask) 
        logits = outputs[0] 
        # obtain softmax probabilities 
        prob = nnf.softmax(logits, dim=1).flatten() 
        probabilities_gpt2.append(prob) 

    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities_roberta)): 
        for j in range(46): 
            prob_sum[j] += probabilities_roberta[i][j] 
    for i in range(len(probabilities_electra)):  
        for j in range(46): 
            prob_sum[j] += probabilities_electra[i][j]  
    for i in range(len(probabilities_gpt2)):  
        for j in range(46):  
            prob_sum[j] += probabilities_gpt2[i][j] 
            
    prob_sum /= (3 * len(probabilities_gpt2)) 
    
    predicted_class = np.argmax(prob_sum)
    
    predicted_classes.append(predicted_class)


  7%|▋         | 3140/43576 [33:39<7:42:25,  1.46it/s] Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (565 > 512). Running this sequence through the model will result in indexing errors


Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


  9%|▊         | 3737/43576 [39:51<5:44:37,  1.93it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 20%|█▉        | 8594/43576 [1:30:10<6:43:00,  1.45it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 27%|██▋       | 11626/43576 [2:01:14<4:35:27,  1.93it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 29%|██▊       | 12438/43576 [2:09:39<4:14:52,  2.04it/s]

Long Text!! Using Head+Tail Truncation


 40%|████      | 17522/43576 [3:02:40<4:30:02,  1.61it/s] 

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 56%|█████▌    | 24334/43576 [4:19:19<4:29:11,  1.19it/s] 

In [ ]:
submission.iloc[:,1] = predicted_classes 

submission.to_csv("GPT2_Electra_roberta.csv",index=False) 

In [ ]:
submission